# Returns engine demo

Demonstrates usage of the returns engine, which consumes a backend db with "time", "symbol" and fair expressions

In [1]:
import mnemosyne as ms
from mnemosyne.engines import ReturnsEngine 
from datetime import datetime as Datetime, date as Date
from timedelta_isoformat import timedelta as Timedelta
import polars as pl
from pathlib import Path
from tqdm.auto import tqdm

# Minimal example

In [2]:
symbol_enum = pl.Enum(['BTC', 'ETH', 'BTCDOWN', 'BNB', 'SHIB'])

query_lf = pl.DataFrame({
    'symbol': ['BTC', 'ETH', 'BTCDOWN', 'BNB', 'SHIB'], 
    'times': [Datetime(2025, 8, 1, 9, 0, 0)] * 5
}).with_columns(pl.col('symbol').cast(symbol_enum)).lazy()

backend_db = ms.binance.BinanceLastTradesGrid(
            peg_symbol='USDC', 
            grid_interval=Timedelta(seconds=5), 
            dataset_type=ms.DatasetType.BinanceSpotTrades, 
        ).lazyframe()

re = ReturnsEngine(
    backend_db, 
    backend_fair_expr = pl.col('vwap_price'), 
    backend_time_expr = pl.col('last_event_time')
)

value = re.query(query_lf, 
    start_time_expr = pl.col('times'), 
    mark_duration=Timedelta(minutes=10), 
    tick_lag_tolerance=Timedelta(minutes=10), 
    verbose_debug=False
)
value.collect()

sys:1: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided


symbol,times,max_tick_to_query_lag,return
enum,datetime[μs],duration[μs],f64
"""BTC""",2025-08-01 09:00:00,3s 766592µs,0.00313
"""ETH""",2025-08-01 09:00:00,399614µs,0.004187
"""BTCDOWN""",2025-08-01 09:00:00,null,null
"""BNB""",2025-08-01 09:00:00,1s 88569µs,0.003798
"""SHIB""",2025-08-01 09:00:00,5s 547004µs,0.00332


In [3]:
import hvplot.polars
import hvplot 
hvplot.extension('plotly')


save_root = Path('~/Documents/binance_gridded_returns')

pivot_dates = [
    Date(2022, 1, 1),
    Date(2023, 2, 1),
    Date(2024, 1, 1),
    Date(2025, 1, 1),
    Date(2026, 1, 1)
]

mark_exprs = {
    'now_to_p10m': (pl.col('time'), Timedelta(minutes=10)),
    'p1m_to_p11m': (pl.col('time') + Timedelta(minutes=1), Timedelta(minutes=10)),
    'm10m_to_now': (pl.col('time') - Timedelta(minutes=10), Timedelta(minutes=10)),
    'm20m_to_now': (pl.col('time') - Timedelta(minutes=20), Timedelta(minutes=20)),
    'm30m_to_now': (pl.col('time') - Timedelta(minutes=30), Timedelta(minutes=30)),
}

# Collect 10m-gridded data

In [4]:
backend_dataset = ms.binance.BinanceLastTradesGrid(
            peg_symbol='USDT', 
            grid_interval=Timedelta(seconds=5), 
            dataset_type=ms.DatasetType.BinanceSpotTrades)
backend_db = backend_dataset.lazyframe()

query_lf = ms.binance.BinanceLastTradesGrid(
            peg_symbol='USDT', 
            grid_interval=Timedelta(minutes=10), 
            dataset_type=ms.DatasetType.BinanceSpotTrades).lazyframe()
            
backend_db = backend_dataset.lazyframe()

re = ReturnsEngine(
    backend_db, 
    backend_fair_expr = pl.col('vwap_price'), 
    backend_time_expr = pl.col('last_event_time')
)

In [5]:
for j in tqdm(range(len(pivot_dates) - 1)):
    query_slice = query_lf.filter(pl.col('date').is_between(pivot_dates[j], pivot_dates[j+1], closed='left'))
    value_lf = re.query_batch(
        query_slice.sort('symbol', 'last_event_time'),
        mark_exprs=mark_exprs,
        tick_lag_tolerance=Timedelta(minutes=10),
        append_lag=False)
    value_lf.sink_parquet(save_root / f'binance_spot_10m_grid_mark10m_{j}.parquet', compression='brotli')

  0%|          | 0/4 [00:00<?, ?it/s]

/tmp/ipykernel_530071/1417076761.py:8: UserWarning:

Sortedness of columns cannot be checked when 'by' groups provided



In [6]:
universe_df = pl.scan_parquet(save_root / 'binance_spot_10m_grid_mark10m_*.parquet').collect()
print(universe_df.shape)
universe_df.group_by('date').agg(pl.col('symbol').unique().len()).sort('date').plot.line(x='date', y='symbol')

(73211214, 21)


alt.Chart(...)

In [7]:
universe_df.sort('time').filter(pl.col('date') == Date(2025, 5, 11)).filter(pl.col('symbol') == 'BTC')

symbol,date,time,peg_symbol,open,high,low,close,volume,trade_count,last_event_time,taker_buy_volume,taker_sell_volume,vwap_taker_buy,vwap_taker_sell,vwap_price,return_now_to_p10m,return_p1m_to_p11m,return_m10m_to_now,return_m20m_to_now,return_m30m_to_now
enum,date,datetime[μs],str,f64,f64,f64,f64,f64,u64,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BTC""",2025-05-11,2025-05-11 00:10:00,"""USDT""",104809.53,104972.0,104655.84,104807.14,309.85195,69035,2025-05-11 00:09:59.903615,160.17704,149.67491,104840.081375,104863.212064,104851.254723,-0.003175,-0.003697,-0.00009,0.00195,0.002392
"""BTC""",2025-05-11,2025-05-11 00:20:00,"""USDT""",104807.14,104811.72,104333.01,104467.35,272.34659,60137,2025-05-11 00:19:59.979340,104.09577,168.25082,104550.356842,104569.926728,104562.446765,-0.005096,-0.002092,-0.003175,-0.003265,-0.001231
"""BTC""",2025-05-11,2025-05-11 00:30:00,"""USDT""",104467.36,104576.25,103926.47,103926.47,257.03953,57406,2025-05-11 00:29:59.996204,109.49165,147.54788,104341.145367,104282.907102,104307.714974,0.001918,0.000696,-0.005096,-0.008255,-0.008344
"""BTC""",2025-05-11,2025-05-11 00:40:00,"""USDT""",103926.48,104365.08,103926.48,104134.43,210.17262,46059,2025-05-11 00:39:59.956333,99.925,110.24762,104214.277978,104223.548055,104219.140667,0.004288,0.00309,0.001918,-0.003187,-0.006352
"""BTC""",2025-05-11,2025-05-11 00:50:00,"""USDT""",104134.44,104585.08,104123.58,104585.05,202.44225,32325,2025-05-11 00:49:59.914662,146.11504,56.32721,104407.967686,104416.555256,104410.357078,-0.001457,-0.002699,0.004288,0.006214,0.001087
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""BTC""",2025-05-11,2025-05-11 23:20:00,"""USDT""",103924.49,103997.6,103818.86,103952.38,85.53112,19869,2025-05-11 23:19:59.801496,42.27725,43.25387,103916.19682,103911.024208,103913.580983,0.000278,0.0003,0.000268,0.001529,0.00235
"""BTC""",2025-05-11,2025-05-11 23:30:00,"""USDT""",103952.38,104015.88,103814.36,103981.28,60.22206,17170,2025-05-11 23:29:59.348348,31.30052,28.92154,103940.057778,103906.115094,103923.756863,-0.001378,-0.00138,0.000278,0.000547,0.001808
"""BTC""",2025-05-11,2025-05-11 23:40:00,"""USDT""",103981.29,103981.29,103826.03,103837.97,53.05682,14255,2025-05-11 23:39:59.507288,27.69482,25.362,103871.780825,103898.185123,103884.402497,0.001237,0.001368,-0.001378,-0.001101,-0.000832


# Perp data

In [ ]:
backend_dataset = ms.binance.BinanceLastTradesGrid(
            peg_symbol='USDT', 
            grid_interval=Timedelta(seconds=5), 
            dataset_type=ms.DatasetType.BinanceUmPerpTrades)
backend_db = backend_dataset.lazyframe()

# query_lf = backend_dataset.cast_symbol_col_to_enum(backend_dataset.universe()).with_columns(
#         time=pl.datetime_ranges(
#             pl.col('date'), 
#             pl.col('date').dt.offset_by('1d'),
#             interval=grid_query_interval,
#             closed='left'
#         )
#     ).explode('time').lazy()
query_lf = ms.binance.BinanceLastTradesGrid(
            peg_symbol='USDT', 
            grid_interval=Timedelta(minutes=10), 
            dataset_type=ms.DatasetType.BinanceSpotTrades).lazyframe()

re = ReturnsEngine(
    backend_db, 
    backend_fair_expr = pl.col('vwap_price'), 
    backend_time_expr = pl.col('last_event_time')
)

In [ ]:
for j in tqdm(range(len(pivot_dates) - 1)):
    query_slice = query_lf.filter(pl.col('date').is_between(pivot_dates[j], pivot_dates[j+1], closed='left'))
    value_lf = re.query_batch(
        query_slice.sort('symbol', 'last_event_time'),
        mark_exprs=mark_exprs,
        tick_lag_tolerance=Timedelta(minutes=10),
        append_lag=False)
    value_lf.sink_parquet(save_root / f'binance_futures_10m_grid_mark10m_{j}.parquet', compression='brotli')

In [10]:
universe_df = pl.scan_parquet(save_root / 'binance_futures_10m_grid_mark10m_*.parquet').collect()
print(universe_df.shape)
universe_df.group_by('date').agg(pl.col('symbol').unique().len()).sort('date').plot.line(x='date', y='symbol')
universe_df

(73211214, 21)


symbol,date,time,peg_symbol,open,high,low,close,volume,trade_count,last_event_time,taker_buy_volume,taker_sell_volume,vwap_taker_buy,vwap_taker_sell,vwap_price,return_now_to_p10m,return_p1m_to_p11m,return_m10m_to_now,return_m20m_to_now,return_m30m_to_now
enum,date,datetime[μs],str,f64,f64,f64,f64,f64,u64,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1INCH""",2022-01-01,2022-01-01 06:00:00,"""USDT""",2.436,2.437,2.428,2.432,20163.3,133,2022-01-01 05:59:57.916,7724.2,12439.1,2.430749,2.432594,2.431887,0.001702,0.000872,-0.001429,-0.004999,-0.00341
"""1INCH""",2022-01-01,2022-01-01 06:20:00,"""USDT""",2.437,2.444,2.433,2.436,33396.7,161,2022-01-01 06:19:57.979,27846.1,5550.6,2.439286,2.437145,2.43893,-0.003591,-0.002636,0.000184,0.001886,0.000454
"""1INCH""",2022-01-01,2022-01-01 10:40:00,"""USDT""",2.407,2.411,2.396,2.41,62915.1,1000,2022-01-01 10:39:58.541,29575.2,33339.9,2.401732,2.402023,2.401886,0.002977,0.002568,0.000954,-0.006091,-0.010355
"""1INCH""",2022-01-02,2022-01-02 03:30:00,"""USDT""",2.5,2.508,2.5,2.508,6451.8,74,2022-01-02 03:29:58.546,3645.8,2806.0,2.503042,2.504679,2.503754,0.001761,0.001227,0.003704,0.000434,0.00333
"""1INCH""",2022-01-03,2022-01-03 14:50:00,"""USDT""",2.621,2.64,2.604,2.607,268542.5,2899,2022-01-03 14:49:58.294,149138.9,119403.6,2.62326,2.623592,2.623408,-0.0026,0.001279,-0.005254,-0.001479,0.004717
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""BURGER""",2025-03-27,2025-03-27 10:40:00,"""USDT""",0.0472,0.0475,0.0469,0.0474,191600.0,156,2025-03-27 10:39:27.497752,98131.4,93468.6,0.047312,0.047048,0.047183,null,null,null,null,null
"""BURGER""",2025-03-27,2025-03-27 13:20:00,"""USDT""",0.0447,0.0447,0.0435,0.0435,180224.6,147,2025-03-27 13:19:52.865307,60586.2,119638.4,0.044306,0.04439,0.044362,null,null,null,null,null
"""BURGER""",2025-03-27,2025-03-27 16:00:00,"""USDT""",0.034,0.034,0.0333,0.0337,958765.4,220,2025-03-27 15:59:50.576751,879476.1,79289.3,0.033965,0.033769,0.033949,null,null,null,null,null


# Test cells: batch & separate query should be equivalent

In [ ]:
# Use short names - 'return_' prefix will be added automatically by query_batch
appends = [
    (Timedelta(0), Timedelta(minutes=10), 'now_to_p10m'),
    (Timedelta(minutes=1), Timedelta(minutes=10), 'p1m_to_p11m'),
    (Timedelta(minutes=-10), Timedelta(minutes=10), 'm10m_to_now'),
    (Timedelta(minutes=-20), Timedelta(minutes=20), 'm20m_to_now'),
    (Timedelta(minutes=-30), Timedelta(minutes=30), 'm30m_to_now'),
]

# OLD get_returns_col function for single queries (kept for compatibility)
# Returns a dataframe with one column corresponding to the given returns
def get_returns_col(re, query_lf, start_offset, mark_offset, col_name, **kwargs) -> pl.LazyFrame:
    return re.query(query_lf,
            start_time_expr = pl.col('time') + start_offset, 
            mark_duration=mark_offset, 
            tick_lag_tolerance=Timedelta(minutes=10), 
            # append_lag=False,
            append_lag=True, 
            append_query_tick_times=True, 
            append_start_end_fairs=True, 
            **kwargs
        ).select(
            pl.col('return').alias(f'return_{col_name}'), 
            pl.col('^.*query_time$').name.suffix(f'_{col_name}'), 
            pl.col('^.*_fair$').name.suffix(f'_{col_name}'))

In [ ]:
query_lf = ms.binance.BinanceLastTradesGrid(
    peg_symbol='USDT',
    grid_interval=Timedelta(minutes=10),
    dataset_type=ms.DatasetType.BinanceUmPerpTrades, 
).lazyframe()

backend_db = ms.binance.BinanceLastTradesGrid(
            peg_symbol='USDT', 
            grid_interval=Timedelta(seconds=4), 
            dataset_type=ms.DatasetType.BinanceUmPerpTrades, 
        ).lazyframe()

re = ReturnsEngine(
    backend_db, 
    backend_fair_expr = pl.col('vwap_price'), 
    backend_time_expr = pl.col('last_event_time')
)

In [ ]:
# OLD APPROACH: Multiple separate queries
query_lf_slice = query_lf.filter(pl.col('date') >= Date(2025, 10, 20))

benchmark_df = pl.concat([
    query_lf_slice, 
    *[get_returns_col(re, query_lf_slice, *a) for a in appends]
], how='horizontal').sort('symbol', 'time').collect()
benchmark_df

In [ ]:
# NEW APPROACH: Single batch query - much faster!
query_lf_slice = query_lf.filter(pl.col('date') >= Date(2025, 10, 20))

# Convert appends to mark_exprs for query_batch
mark_exprs = {
    'now_to_p10m': (pl.col('time'), Timedelta(minutes=10)),
    'p1m_to_p11m': (pl.col('time') + Timedelta(minutes=1), Timedelta(minutes=10)),
    'm10m_to_now': (pl.col('time') - Timedelta(minutes=10), Timedelta(minutes=10)),
    'm20m_to_now': (pl.col('time') - Timedelta(minutes=20), Timedelta(minutes=20)),
    'm30m_to_now': (pl.col('time') - Timedelta(minutes=30), Timedelta(minutes=30)),
}

answer_df = re.query_batch(
    query_lf_slice,
    mark_exprs=mark_exprs,
    tick_lag_tolerance=Timedelta(minutes=10),
    # append_lag=False
    append_lag=True, 
    append_query_tick_times=True, 
    append_start_end_fairs=True
).sort('symbol', 'time').collect()

# Verify same column names as old approach
print(f"Column names: {answer_df.columns}")
answer_df

In [ ]:
# Double-check: this should be all true if not null!!
(answer_df.select('^return_.*$') == benchmark_df.select('^return_.*$'))

# Debugging script

Used during dev & debugging

In [ ]:
mark_exprs, append_cols, query_lf_withidx = re.query_batch(
    query_lf_slice,
    mark_exprs=mark_exprs,
    tick_lag_tolerance=Timedelta(minutes=10),
    # append_lag=False
    append_lag=True, 
    append_query_tick_times=True, 
    append_start_end_fairs=True
)

schema = append_cols.collect_schema()
metric_cols = [c for c in schema.names() if c not in ['row_id', 'return_col_name']]

wide_append_cols = None
for ret_col_name in mark_exprs.keys():
    filtered = (
        append_cols
        .filter(pl.col('return_col_name') == ret_col_name)
        .select(
            'row_id',
            *[pl.col(col).alias(f'{col}_{ret_col_name}') for col in metric_cols]
        )
    )

    if wide_append_cols is None:
        wide_append_cols = filtered
    else:
        wide_append_cols = wide_append_cols.join(filtered, on='row_id', how='left')



# Join back to original query
result = query_lf_withidx.join(
    wide_append_cols, on='row_id',
    how='left'
).drop('row_id')

answer_df = result.sort('symbol', 'time').collect()

append_cols.collect()